# Analysis
We first load the data saved in the preprocessing steps.

In [ ]:
import pandas as pd
import os
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from plot_maps import single_map,map_with_progress_bar
import helpers
import statsmodels.api as sm
from settings import data_folder, preprocessed_folder

%load_ext autoreload
%autoreload 2

helpers.set_plotting() 

In [ ]:
measure_name = [
    "Envelope \nrenovation (M-01)",
    "Wood/pellets\nheating (M-02)",
    "Wood heating\n <70kW (M-03)",
    "Wood heating\n >70kW (M-04)",
    "Air/water heat \npump (M-05)",
    "Elec. heat \npump (M-06)",
    "Connection to dec.\n heating (M-07)",
]

db_with_terrain_class = helpers.load_database()


# Which Population benefits most from subsidies? 

In this section, we attempt to characterize the persons that are asking for subsidies. Since we don't have any information about the persons carrying out the renovations, we use the information at the municipality level.

### First approach
We use a t-test to compare the population to the national average.

In [ ]:
# Features that are used
columns_pop_group = [
    "Population - Part du groupe d'âge 20-64 ans",
    "Population - Part du groupe d'âge 65+ ans",
    "Population - Taille moyenne des ménages",
    "Population - Etrangers",
    "Taux d'aide sociale",
]
combined = helpers.add_socio_economic_features(db_with_terrain_class)
combined


In [ ]:
socio_features_avg_ch = helpers.get_economic_features("CH")
fig, ax = plt.subplots(figsize=(10, 10))
tmp = combined[columns_pop_group]
mean_ch = socio_features_avg_ch[columns_pop_group].to_numpy().reshape(-1)
# Compute t-test
t_test = (tmp.mean() - mean_ch) / tmp.std()
sns.barplot(
    data=t_test.to_frame("value").reset_index(),
    x="index",
    y="value",
    order=columns_pop_group,
    ax=ax,
)
helpers.set_label(xlabel="Variable", ylabel="T-test")
plt.xticks(rotation=70)
helpers.save("pop_subside")


We compute the t-test for each measure.

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))
tmp = combined[columns_pop_group + ["Nr. HFM 2015"]].groupby("Nr. HFM 2015")
mean_ch = socio_features_avg_ch[columns_pop_group].to_numpy()
t_test = (tmp.mean() - mean_ch) / tmp.std()
t_test = t_test.loc[["M-01", "M-02", "M-03", "M-04", "M-05", "M-06", "M-07"]]
sns.barplot(
    data=pd.melt(t_test, ignore_index=False).reset_index(),
    x="Nr. HFM 2015",
    y="value",
    hue="variable",
    hue_order=columns_pop_group,
    ax=ax,
)
helpers.set_label(ylabel="T-test")
ax.legend(bbox_to_anchor=(1.01, 0.6))
helpers.save("pop_subside_permeasure")


### Second approach: classification

We approach the problem as a classification problem, where we need to classify two population groups
- the persons who asked for a subsidy,
- the remaining persons (include persons that haven't done any renovations and those who have done a renovation, but didn't ask for subsidies).

**Caveats**: Since we only have information at the municipality (we don't have any info on the persons asking for a subsidy), we aggregate the data at the municipality level. An inherent hypothesis is that the population of a given municipality share all the same profile. Also we assume that the features of the building does not impact the decision of getting or not a subsidy.

The regBL is used to get the whole list of buildings in Switzerland. We only consider the buildings that are used as houses purposes 

In [ ]:
# Load the regbl (all the buildings in CH)
regbl = pd.read_pickle(os.path.join(preprocessed_folder, "rebgl.pickle")).astype(
    {"EGID": "Int64"}
)

# Repeat some rows since some values of the EGID column of the db_with_terrain_class database contains multiple EGID in the same cell.
db_with_terrain_class_exploded = helpers.explode_db(db_with_terrain_class)

# Combine regbl with our database and identify which buildings have done which measure.
combined_regbl = pd.merge(regbl, db_with_terrain_class_exploded, on="EGID", how="left")

combined_regbl = helpers.add_renov_indicator(combined_regbl)
hab_to_keep = [
    "Usage d'habitation",
    "Maison avec usage annexe",
    "Part. à usage d'hab.",
    np.nan,
]
# Filter the data by keeping only existing buildings and those with habitation purpose
combined_regbl_sub = combined_regbl[
    (combined_regbl.Statut_bat == "existant")
    & (combined_regbl.Cat_bat.isin(hab_to_keep))
].copy()

In [ ]:
use_rooms=False
measure_columns=['renov', 'Envelope_renov',
       'Heating_renov', 'M-01', 'M-02', 'M-03', 'M-04', 'M-05', 'M-06',
       'M-07', 'M-08', 'M-09', 'M-10', 'M-11', 'M-12', 'M-13', 'M-14', 'M-15',
       'M-16', 'M-17', 'M-18',"ones"]
if use_rooms:
    combined_regbl_sub=combined_regbl_sub.dropna(subset="WAZIM").reset_index(drop=True)
    combined_regbl_sub[measure_columns]*=combined_regbl_sub.WAZIM.to_numpy().reshape(-1,1)

In [ ]:
# Load economic features and add if the municipalitiy is alpine and the topology
hab_alpin_typ = helpers.add_alpin_topo(
    helpers.get_economic_features("commune"), "BFS_NUMMER"
)
hab_alpin_typ


Aggregate data/municipality

In [ ]:
combined_regbl_sub["Bois"] = combined_regbl_sub[["M-02", "M-03", "M-04"]].max(axis=1)
combined_regbl_sub["Heatpump"] = combined_regbl_sub[["M-05", "M-06"]].max(axis=1)
combined_regbl_sub["Epoque_constr_agg"] = combined_regbl_sub.Epoque_constr.replace(
    {
        "Avant 1919": "<1960",
        "1919-1945": "<1960",
        "1971-1980": "1961-2000",
        "1946-1960": "<1960",
        "1961-1970": "1961-2000",
        "1986-1990": "1961-2000",
        "1996-2000": "1961-2000",
        "1981-1985": "1961-2000",
        "2006-2010": ">2000",
        "1991-1995": "1961-2000",
        "2001-2005": ">2000",
        "> 2015": ">2000",
        "2011-2015": ">2000",
    }
)
only_individual = False  # To change to True if only individual buildings
if only_individual:
    combined_regbl_sub_ = combined_regbl_sub[
        combined_regbl_sub.Classe_bat.isin(
            ["Maison individuelle", "Maison à 2 logements"]
        )
    ]
else:
    combined_regbl_sub_ = combined_regbl_sub

epoque = None  # To change to "<1960","1961-2000",">2000" to specific buildings
if epoque is None:
    combined_regbl_sub_ = combined_regbl_sub
else:
    assert epoque in ["<1960", "1961-2000", ">2000"]
    combined_regbl_sub_ = combined_regbl_sub[
        combined_regbl_sub["Epoque_constr_agg"] == epoque
    ]
statistics = ["sum"]
prop_subsidy_per_municipality = combined_regbl_sub_.groupby(["BFS_NUMBER"]).agg(
    {
        "renov": statistics,
        "Envelope_renov": statistics,
        "Heating_renov": statistics,
        "M-07": statistics,
        "Bois": statistics,
        "Heatpump": statistics,
        "ones": statistics,
    }
)

nb_renovation = prop_subsidy_per_municipality.xs("sum", level=1, axis=1).drop(columns="ones")
total_building=prop_subsidy_per_municipality[("ones","sum")].to_numpy().reshape(-1,1)
prop_subsidy_per_municipality = pd.merge(
    nb_renovation,
    total_building - nb_renovation,
    left_index=True,
    right_index=True,
)
# Doesn't exist
prop_subsidy_per_municipality = prop_subsidy_per_municipality.drop(
    ["2391", "5391"]
).astype(int)  
prop_subsidy_per_municipality


### Run the models
We first run the model on building envelope renovations 

In [ ]:
y = prop_subsidy_per_municipality[["Envelope_renov_x", "Envelope_renov_y"]]
y.index = y.index.astype("Int64")
cols = [
    "BFS_NUMMER",
    "Population - Habitants",
    "Population - Part du groupe d'âge 65+ ans",
    "Population - Etrangers",
    "Population - Taux brut de natalité",
    "Population - Taille moyenne des ménages",
    "Construction, logement - Nouveaux logements construits",
    "Economie - Emplois dans le secteur primaire",
    "Economie - Emplois dans le secteur secondaire",
    "Economie - Emplois dans le secteur tertiaire",
    "Taux d'aide sociale",
    "Revenu imposable par contribuable, en francs",
    "Alpine",
    "Typology",
]
# Check topology of municipality
X = hab_alpin_typ[cols].set_index("BFS_NUMMER").loc[y.index].copy()
# Center the data with the national average
mean = socio_features_avg_ch[cols[:-2]].drop(columns="BFS_NUMMER")
X = helpers.build_design_matrix(X, mean)

# Possible outliers
# X=X.drop(["2701","351","5409","5194","261","5872","3506","371","6421"])
# y=y.drop(["2701","351","5409","5194","261","5872","3506","371","6421"])

bin = sm.families.Binomial()
glm_binom = sm.GLM(y[y.Envelope_renov_y > 0], X[y.Envelope_renov_y > 0], family=bin)
res = glm_binom.fit()
print(res.summary())


In [ ]:
name_param = [
    "Population share 65+",
    "Foreign share",
    "Birth rate",
    "Average household size",
    "New Buildings",
    "Primary sector employment per capita",
    "Secondary sector employment per capita",
    "Service sector employment per capita",
    "Social subsidy reciever",
    "Average taxable income/Taxpayer",
    "Alpine region",
    "Rural",
    "Urban",
]

helpers.plot_param_glm(res, name_param)
plt.title("Parameters building envelope renovation (M-01)")
helpers.save("glm_m01")


The model is run on heating system replacement (M-02,...,M-07)

In [ ]:
y = prop_subsidy_per_municipality[["Heating_renov_x", "Heating_renov_y"]]
y.index = y.index.astype("Int64")

bin = sm.families.Binomial()
glm_binom = sm.GLM(y[y.Heating_renov_y > 0], X[y.Heating_renov_y > 0], family=bin)
res = glm_binom.fit()
print(res.summary())
helpers.plot_param_glm(res, name_param)
plt.title("Parameters Heating system replacement (M-02,...,M-07)")
helpers.save("glm_m02")



We finally check if the parameters changes if we split the measures into three categories:
- Wood heating (m-02,m-03,m-04)
- Heatpump (m-05,m-06)
- Connection to a decentralised heating system (m-07)

We first start with the wood.

In [ ]:
y = prop_subsidy_per_municipality[["Bois_x", "Bois_y"]]
y.index = y.index.astype("Int64")

bin = sm.families.Binomial()
glm_binom = sm.GLM(y[y.Bois_y > 0], X[y.Bois_y > 0], family=bin)
res = glm_binom.fit()
print(res.summary())
helpers.plot_param_glm(res, name_param)
plt.title("Parameters Wood heating system (M-02,...,M-04)")
helpers.save("glm_m02_m04")


We run the same model on the second category.

In [ ]:
y = prop_subsidy_per_municipality[["Heatpump_x", "Heatpump_y"]]
y.index = y.index.astype("Int64")
bin = sm.families.Binomial()
glm_binom = sm.GLM(y[y.Heatpump_y > 0], X[y.Heatpump_y > 0], family=bin)
res = glm_binom.fit()
print(res.summary())

helpers.plot_param_glm(res, name_param)
plt.title("Parameters Heat pump system (M-05,M-06)")
helpers.save("glm_m05_m06")


We finally run the model on the last category.

In [ ]:
y = prop_subsidy_per_municipality[["M-07_x", "M-07_y"]]
y.index = y.index.astype("Int64")
bin = sm.families.Binomial()
glm_binom = sm.GLM(y[y["M-07_y"] > 0], X[y["M-07_y"] > 0], family=bin)
res = glm_binom.fit()
print(res.summary())

helpers.plot_param_glm(res, name_param)
plt.title("Parameters Dec. heating system (M-07)")
helpers.save("glm_m07")
